In [12]:
from infoseq.bpe import bpe_from_text

from infoseq.dataset import kjv_path

with open(kjv_path, 'rb') as f:
    kjv = f.read()

len(kjv)

4457889

In [13]:
bpe = bpe_from_text(kjv, max_tokens=5000)
len(bpe)

with open('/tmp/bpe.json', 'w') as f:
    f.write(bpe.to_json())

In [9]:
[k for k in bpe.encode_map if len(k) > 1][:10]

[]

In [14]:
with open('/tmp/bpe.dot', 'w') as f:
    f.write(bpe.to_dot())

In [20]:
import os
import json
from infoseq.dataset import DATA_PATH

decode_map = json.load(open(os.path.join(DATA_PATH, 'gpt2-encoder.json')))
encode_map = {v: k for k, v in decode_map.items()}

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [12]:
import os
from infoseq.gpt2 import get_encoder
from infoseq.token import Tokenization

MODELS_PATH = os.path.expanduser('~/code/gpt-2/models')

encoder = get_encoder('1558M', MODELS_PATH)

decode_map = {i: encoder.decode([i]) for i in range(50257)}
encode_map = {v: k for k, v in decode_map.items()}

def get_prev_token(code):
    while code:
        code = code[:-1]
        if code in encode_map:
            return encode_map[code]
    return None

# Generate graphviz dot file
node_lines = ['Root [label=""]']
edge_lines = []
for token, code in sorted(decode_map.items()):
    if token > 0:
        prev_token = get_prev_token(code)
        if prev_token is not None:
            edge_lines.append(f'{prev_token} -> {token}')
        else:
            edge_lines.append(f'Root -> {token}')
    label = repr(code).replace('"','\\"')
    node_lines.append(f'{token} [label="{label}"]')
all_lines = ['digraph {'] + node_lines + edge_lines + ['}']
graphviz = '\n'.join(all_lines)

with open('/tmp/gpt2.dot', 'w') as f:
    f.write(graphviz)

b'hello'